# Домашнее задание 2: _Парсинг Авито_

In [1]:
# для работы с вебом
from bs4 import BeautifulSoup
import requests
import random
import time

# для чистки текста
import re
import html

# для работы с файловой системой
import os

# для составления таблицы с метаинформацией
from datetime import datetime, timedelta
import pandas as pd

## Скачать страницу и обработать её содержимое

In [2]:
reg_text = re.compile("<p>(.*?)<\/p>")
reg_date = re.compile("размещено (.*)?  ")
reg_id = re.compile("№ ([0-9]+)")

In [3]:
def extract_date(ad_metadata):
    ad_date = re.search(reg_date, ad_metadata).group(1)
    if "вчера" in ad_date:
        yesterday = datetime.now() - timedelta(days=1)
        ad_date = re.sub("вчера", yesterday.strftime("%d").lstrip("0") + " октября", ad_date)
    elif "сегодня" in ad_date:
        today = datetime.now()
        ad_date = re.sub("сегодня", today.strftime("%d").lstrip("0") + " октября", ad_date)
    return ad_date

In [4]:
def extract_meta(link, soup):
    page_meta = {
        "item": "",
        "id": "",
        "date": "",
        "seller": "",
        "address": ""
    }
    ad_metadata = soup.select("div.title-info-metadata-item")[0].text
    
    page_meta["item"] = link[46:]
    page_meta["id"] = re.search(reg_id, ad_metadata).group(1)
    page_meta["date"] = extract_date(ad_metadata)
    page_meta["seller"] = soup.find("div", attrs={"class": "seller-info-name"}).find("a").text
    page_meta["address"] = soup.find("span", attrs={"itemtype": "http://schema.org/PostalAddress"}).text
    
    return page_meta

In [5]:
def parse_page(link):
    link_html = requests.get(link).text
    soup = BeautifulSoup(link_html, "lxml")
    ad_title = soup.select("h1.title-info-title")[0].text.strip()
    
    ad_text = list(re.findall(reg_text, link_html))
    raw_text = "\n".join(ad_text)
    
    total_text = ad_title + "\n" + raw_text
    text_meta = extract_meta(link, soup)
    return total_text, text_meta

In [6]:
def clean_text(raw_text):
    reg_tag = re.compile("<.*?>")
    reg_space = re.compile("\s{2,}")
    new_text = html.unescape(raw_text)
    new_text = re.sub("<br>", "\n", new_text)
    new_text = reg_tag.sub("", new_text)
    new_text = reg_space.sub(" ", new_text)
    new_text = new_text.replace("\xa0", " ")
    return new_text

## Сохранить

In [7]:
corpus = "." + os.sep + "Avito_Beauty_Corpus"
if not os.path.exists(corpus):
    os.mkdir(corpus)

In [8]:
def save_page(link, contents):
    item = link[46:]
    page_path = corpus + os.sep + item +  ".txt"
    with open(page_path, "w", encoding="utf-8") as page_file:
        page_file.write(contents)

## Процесс

Сначала отдельно соберём ссылки на страницы с товарами и сохраним их в отдельный файл:


```python
start_link = "https://www.avito.ru/moskva/krasota_i_zdorove"
ready_links = []

for i in range(1, 200):
    # генерируем правильный адрес страницы, откуда будем таскать
    # ссылки на объявления
    if i > 1:
        current_link = start_link + "?p={}".format(i)
    else:
        current_link = start_link
    time.sleep(random.randint(1,20))
    print("Parsing page: {}".format(current_link))
    # забираем ссылки на объявления со страницы
    page = requests.get(current_link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        item_hrefs = [a["href"] for a in soup.select("a.item-description-title-link")]
        print("Found {} items".format(len(item_hrefs)))
        # собираем отдельно
        for item_href in item_hrefs:
            ready_link = "https://www.avito.ru" + item_href
            ready_links.append(ready_link)
    print("Done!")
```

Я уже это сделала, но этот код можно снова перенести в ячейку, и он будет работать. А теперь уже будем обкачивать сами страницы с объявлениями:

In [9]:
# with open("./links.txt", "r", encoding="utf-8") as f:
#     links = [link.strip() for link in f.readlines()]
# print("Начинаем собирать корпус из {} ссылок".format(len(links)))

In [10]:
def page_pipeline(item_link, meta_list, counter):
    sleep_time = random.randint(1, 10)
    print("Sleeping for {} sec.".format(sleep_time))
    time.sleep(sleep_time)
    print("Parsing webpage: {}".format(item_link))
    try:
        raw_text, meta = parse_page(item_link)
        new_text = clean_text(raw_text)
        save_page(item_link, new_text)
        for col in meta:
            meta_list[col].append(meta[col])
        counter += 1
        print("SUCCESS! Parsed {} pages".format(counter))
    except:
        print("FAIL")
    return meta_list, counter

In [11]:
start_link = "https://www.avito.ru/moskva/krasota_i_zdorove"
counter = 0
meta_list = {
    "item": [],
    "id": [],
    "date": [],
    "seller": [],
    "address": []
}

for i in range(1, 200):
    # генерируем правильный адрес страницы, откуда будем таскать
    # ссылки на объявления
    if i > 1:
        current_link = start_link + "?p={}".format(i)
    else:
        current_link = start_link
    time.sleep(random.randint(1,20))
    print("Parsing page: {}".format(current_link))
    # забираем ссылки на объявления со страницы
    page = requests.get(current_link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "lxml")
        item_hrefs = [a["href"] for a in soup.select("a.item-description-title-link")]
        print("Found {} items".format(len(item_hrefs)))
        # скачиваем содержимое
        for item_href in item_hrefs:
            ready_link = "https://www.avito.ru" + item_href
            meta_list, counter = page_pipeline(ready_link, meta_list, counter)

Parsing page: https://www.avito.ru/moskva/krasota_i_zdorove
Found 54 items
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/volosy_dlya_naraschivaniya._slavyanskie._70_sm_942336923
SUCCESS! Parsed 1 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/botoks_dlya_volos_vitaker_bio-rekonstruktsiya_682534774
SUCCESS! Parsed 2 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/ogromnoe_kolichestvo_miniatyur._redkie._vintazhi_1000978749
SUCCESS! Parsed 3 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/prodayu_naturalnye_volosy_v_srezah_693374614
SUCCESS! Parsed 4 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/pampersy_dlya_vzroslyh_1480148129
SUCCESS! Parsed 5 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/guerlain_ambre_eternel_1658438654
SUCCESS! Parsed 6 pag

Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/krovat_funktsionalnaya_ortopedicheskaya_vermeiren_1703783207
SUCCESS! Parsed 55 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kitayskie_fitotampony_1323338707
SUCCESS! Parsed 56 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/termo_odeyalo_dlya_obertyvaniya_novoe_3g.garantii_126867484
SUCCESS! Parsed 57 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/mon_parfum_paloma_picasso_vintazh_1602073975
SUCCESS! Parsed 58 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/pampersy_dlya_vzroslyh_1587088430
SUCCESS! Parsed 59 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/aha_ot_akne_1709422101
SUCCESS! Parsed 60 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/bandazh_950532436
SUCCESS! Parsed

Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kanekalon_dlya_kosdred_afrokos_zi-zi_1535886838
SUCCESS! Parsed 109 pages
Sleeping for 5 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kushetka_skladnaya_garantiya._dostavka_segodnya_1158053243
SUCCESS! Parsed 110 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/shina_shantsa_orlett_bn6-53_1467640849
SUCCESS! Parsed 111 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/lorac_mega_pro_1310323179
SUCCESS! Parsed 112 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/fen_polaris_ion_2000w_1566691305
SUCCESS! Parsed 113 pages
Sleeping for 10 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/uf_lampa_36v_ru_818_1076323179
SUCCESS! Parsed 114 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/parfyumernaya_voda_versace_crystal_noir_163964

Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/volosy_dlya_naraschivaniya._slavyanka._60_sm_707988952
SUCCESS! Parsed 163 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/new2018_paco_rabanne_1_million_lucky_tv_100ml_muzh_1234774794
SUCCESS! Parsed 164 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/manikyurnyy_novyy_nabor_scarlett_pedikyurnyy_883971631
SUCCESS! Parsed 165 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/gibridnaya_lampa_sunuv_mini_2_plus_1608723428
SUCCESS! Parsed 166 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/korset_dlya_beremennyh_1211819680
SUCCESS! Parsed 167 pages
Sleeping for 4 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/vakuumnyy_massazher_1078940172
SUCCESS! Parsed 168 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/

SUCCESS! Parsed 217 pages
Sleeping for 4 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kushetka_s_telezhkoy_i_stulom_master_mastera_1184972986
SUCCESS! Parsed 218 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/chanel_chance_eau_tendre_1130490376
SUCCESS! Parsed 219 pages
Sleeping for 4 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/dior_este_lauder_1224617245
SUCCESS! Parsed 220 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/parfyumernaya_voda_serge_lutens_leau_de_paille_936914689
SUCCESS! Parsed 221 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/britva_norelko_996070425
SUCCESS! Parsed 222 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kosmetichka-_klatch_filorga_novaya_640360386
SUCCESS! Parsed 223 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_

...

SUCCESS! Parsed 9990 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/meditsina_dlya_bolnogo_731603174
SUCCESS! Parsed 9991 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/pomady_tom_ford_1396502285
SUCCESS! Parsed 9992 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/givenchy_live_irresistible_tual.voda_50_ml_1372646162
SUCCESS! Parsed 9993 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/lampa_1638791619
SUCCESS! Parsed 9994 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/podguzniki_822138802
SUCCESS! Parsed 9995 pages
Sleeping for 10 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/massazher_ruchnoy_1015099087
SUCCESS! Parsed 9996 pages
Sleeping for 4 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/elektronnye_detskie_vesy_1485449015
SUCCESS! Par

Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/syvorotka_mission_perfection_clarins_1615423810
SUCCESS! Parsed 10046 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/novyy_blender_bosch_1407280544
SUCCESS! Parsed 10047 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/uryazh_uriage_1573599932
SUCCESS! Parsed 10048 pages
Sleeping for 5 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/podushka_na_sidene_anatomicheskaya_ormax_995403653
SUCCESS! Parsed 10049 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/cigar_sommander_60_ml_1556667148
SUCCESS! Parsed 10050 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/volosy_dlya_naraschivaniya_575546701
SUCCESS! Parsed 10051 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/ultrafioletovaya_lampa_1080654407
SUCCESS! Par

SUCCESS! Parsed 10098 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/nabor_kosmetiki_1608505575
SUCCESS! Parsed 10099 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/prodam_invalidnoe_kreslo-katalka_986989980
SUCCESS! Parsed 10100 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/tom_ford_teni_1415983980
SUCCESS! Parsed 10101 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/breg_fusion_castom_xs_pravyy_1148000131
SUCCESS! Parsed 10102 pages
Sleeping for 6 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/tualetnaya_voda_mugler_alien_1240944081
SUCCESS! Parsed 10103 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kosmetika_1578839191
SUCCESS! Parsed 10104 pages
Sleeping for 5 sec.
Parsing webpage: https://www.avito.ru/moskva_zelenograd/krasota_i_zdorove/aromat_givenchy_

SUCCESS! Parsed 10151 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/marc_jacobs_daisy_dream_twinkle_1489540393
SUCCESS! Parsed 10152 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/avon_eve_duet_for_her_50ml_1414585002
SUCCESS! Parsed 10153 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/palitra_teney_1538155949
SUCCESS! Parsed 10154 pages
Sleeping for 2 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/upakovka_30_sht._pampersy_dlya_vzroslyh_seni_6_kapel_1221806617
SUCCESS! Parsed 10155 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/ochki_novye_1641394446
SUCCESS! Parsed 10156 pages
Sleeping for 3 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/prodam_domashniy_solyariy_philips_original_home_sola_1010683138
SUCCESS! Parsed 10157 pages
Sleeping for 7 sec.
Parsing webpage: https://www.

Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/duhi_lacoste_1454096116
SUCCESS! Parsed 10206 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/dolcegabbana_3-limperatrice_100_ml_1441871827
SUCCESS! Parsed 10207 pages
Sleeping for 9 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/duhi_vintazh_magie_noire_ot_lancome_15ml_1978_g_552711919
SUCCESS! Parsed 10208 pages
Sleeping for 1 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/naso_di_raza_tester_original_100_ml_1679385076
SUCCESS! Parsed 10209 pages
Sleeping for 8 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/kateter_foleya_2-h_hodovoy_1183890108
SUCCESS! Parsed 10210 pages
Sleeping for 7 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/test_poloski_van_tach_ultra_1455367299
SUCCESS! Parsed 10211 pages
Sleeping for 10 sec.
Parsing webpage: https://www.avito.ru/moskva/krasota_i_zdorove/prodam_test-polosk

## Обработка метаданных

In [13]:
items_df = pd.DataFrame(meta_list)
items_df.set_index("id", inplace=True)
items_df["seller"] = items_df["seller"].apply(clean_text)
items_df[:10]

,item,date,seller,address
id,,,,
942336923,volosy_dlya_naraschivaniya._slavyanskie._70_sm...,20 октября в 18:07,Наталимаркет,м. Тульская
682534774,botoks_dlya_volos_vitaker_bio-rekonstruktsiya_...,20 октября в 17:06,Наталимаркет,м. Тульская
1000978749,ogromnoe_kolichestvo_miniatyur._redkie._vintaz...,20 октября в 15:42,Юлия,м. Бауманская
693374614,prodayu_naturalnye_volosy_v_srezah_693374614,20 октября в 15:24,Владимир,м. Котельники
1480148129,pampersy_dlya_vzroslyh_1480148129,20 октября в 22:49,Анастасия,м. Медведково
1658438654,guerlain_ambre_eternel_1658438654,20 октября в 22:46,Вера,м. Площадь революции
1461463817,volosy_naturalnye_1461463817,20 октября в 22:43,Ирина,м. Площадь революции
1026532024,montale_oudmazing_50_ml_1026532024,20 октября в 22:44,Яна,м. Первомайская
1090392049,nakolenniki_shock_doctor_m_1090392049,20 октября в 22:46,Suriko Kirill,м. Полянка


In [15]:
print(len(items_df))
print(len(os.listdir(corpus)))

10215
5474


In [16]:
items_df.to_csv("./items_meta.csv", sep=";")